In [1]:
import pandas as pd
import re, string, unicodedata
import nltk
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import gc
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split, cross_val_score
import xgboost as xgb

In [2]:
#Loading the data
df_data = pd.read_csv('C:\Sujata\CompScience\Winter 2021\Data Mining\Project Final\mbti_1\mbti_1.csv')

# Removed |||
df_data.posts = df_data.posts.replace(r'[|||]', r' ', regex=True)

c:\users\madhu\appdata\local\programs\python\python37\lib\site-packages\pandas\core\dtypes\inference.py:178: FutureWarning: Possible set union at position 2
  re.compile(obj)


In [3]:
#Diving the 16 MBTI type to only 4
df_data['is_E'] = df_data['type'].apply(lambda x: 1 if x[0] == 'E' else 0)
df_data['is_S'] = df_data['type'].apply(lambda x: 1 if x[1] == 'S' else 0)
df_data['is_T'] = df_data['type'].apply(lambda x: 1 if x[2] == 'T' else 0)
df_data['is_J'] = df_data['type'].apply(lambda x: 1 if x[3] == 'J' else 0)

In [4]:
#dropping the type column
df_data = df_data.drop(['type'], axis=1)

In [5]:
#Separating the data according to I/E , N/S, F/T AND P/J. All of them has the post column
df_data_IE = df_data.drop(['is_S', 'is_T', 'is_J'], axis=1)
df_data_NS = df_data.drop(['is_E', 'is_T', 'is_J'], axis=1)
df_data_FT = df_data.drop(['is_S', 'is_E', 'is_J'], axis=1)
df_data_PJ = df_data.drop(['is_S', 'is_T', 'is_E'], axis=1)

In [6]:
#Separating into training, validation and Testing data 60, 20, 20 - 5205, 1735 and 1735
train_IE, validate_IE, test_IE = np.split(df_data_IE.sample(frac=1, random_state=42), [int(.6*len(df_data_IE)), int(.8*len(df_data_IE))])
train_NS, validate_NS, test_NS = np.split(df_data_NS.sample(frac=1, random_state=42), [int(.6*len(df_data_NS)), int(.8*len(df_data_NS))])
train_FT, validate_FT, test_FT = np.split(df_data_FT.sample(frac=1, random_state=42), [int(.6*len(df_data_FT)), int(.8*len(df_data_FT))])
train_PJ, validate_PJ, test_PJ = np.split(df_data_PJ.sample(frac=1, random_state=42), [int(.6*len(df_data_PJ)), int(.8*len(df_data_PJ))])

In [7]:
#Split the data to X and Y i.e. posts and labels
#For IE labels
X_train_IE = pd.DataFrame(train_IE['posts'])
Y_train_IE = pd.DataFrame(train_IE['is_E'])
X_validate_IE = pd.DataFrame(validate_IE['posts'])
Y_validate_IE = pd.DataFrame(validate_IE['is_E'])
X_test_IE = pd.DataFrame(test_IE['posts'])
Y_test_IE = pd.DataFrame(test_IE['is_E'])

#For IE labels
X_train_NS = pd.DataFrame(train_NS['posts'])
Y_train_NS = pd.DataFrame(train_NS['is_S'])
X_validate_NS = pd.DataFrame(validate_NS['posts'])
Y_validate_NS = pd.DataFrame(validate_NS['is_S'])
X_test_NS = pd.DataFrame(test_NS['posts'])
Y_test_NS = pd.DataFrame(test_NS['is_S'])

#For IE labels
X_train_FT = pd.DataFrame(train_FT['posts'])
Y_train_FT = pd.DataFrame(train_FT['is_T'])
X_validate_FT = pd.DataFrame(validate_FT['posts'])
Y_validate_FT = pd.DataFrame(validate_FT['is_T'])
X_test_FT = pd.DataFrame(test_FT['posts'])
Y_test_FT = pd.DataFrame(test_FT['is_T'])

#For IE labels
X_train_PJ = pd.DataFrame(train_PJ['posts'])
Y_train_PJ = pd.DataFrame(train_PJ['is_J'])
X_validate_PJ = pd.DataFrame(validate_PJ['posts'])
Y_validate_PJ = pd.DataFrame(validate_PJ['is_J'])
X_test_PJ = pd.DataFrame(test_PJ['posts'])
Y_test_PJ = pd.DataFrame(test_PJ['is_J'])

#display(Y_train_IE.iloc[1])

In [8]:
#Cleaning the data Round 1
def clean_text_round1(text):
    '''Make text lowercase, remove punctuation, remove words containing numbers, remove URL, remove @, '''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('[^a-zA-Z\s]', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'@\w+', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

X_train_IE_clean = pd.DataFrame(X_train_IE.posts.apply(round1))
X_validate_IE_clean = pd.DataFrame(X_validate_IE.posts.apply(round1))
X_test_IE_clean = pd.DataFrame(X_test_IE.posts.apply(round1))

X_train_NS_clean = pd.DataFrame(X_train_NS.posts.apply(round1))
X_validate_NS_clean = pd.DataFrame(X_validate_NS.posts.apply(round1))
X_test_NS_clean = pd.DataFrame(X_test_NS.posts.apply(round1))

X_train_FT_clean = pd.DataFrame(X_train_FT.posts.apply(round1))
X_validate_FT_clean = pd.DataFrame(X_validate_FT.posts.apply(round1))
X_test_FT_clean = pd.DataFrame(X_test_FT.posts.apply(round1))

X_train_PJ_clean = pd.DataFrame(X_train_PJ.posts.apply(round1))
X_validate_PJ_clean = pd.DataFrame(X_validate_PJ.posts.apply(round1))
X_test_PJ_clean = pd.DataFrame(X_test_PJ.posts.apply(round1))

In [9]:
# Tokenization - this is taken from SpaCy  

re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
#stemmer = LancasterStemmer() # did not work

def tokenize(s):
    tokens = re_tok.sub(r' \1 ', s).split()
    return tokens

# Defining lemmatization
wrl = WordNetLemmatizer()

In [10]:
#Remove stopword and also added the MBTI types to it
stop = set(stopwords.words('english'))
stop.update(['infp','infj','intp','intj','entp','enfp','istp','isfp','entj','istj','enfj','isfj','estp','esfp','esfj','estj'])

def remove_stopwords(row):
    return [t for t in row if t not in stop]

In [11]:
#GloVe embedding loading
from tqdm import tqdm

embeddings_index_glove = {}
f = open('C:/Sujata/CompScience/Winter 2021/Data Mining/Project Final/glove.6B.300d.txt', encoding='utf-8')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
    except:
        continue
    embeddings_index_glove[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index_glove))

400001it [00:34, 11703.09it/s]

Found 400001 word vectors.


In [12]:
def sent2vec(s, embeddings_index):
    words = str(s)
    words = tokenize(words)
    words = remove_stopwords(words)
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        w = wrl.lemmatize(w)
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [13]:
gc.collect()

# Create sentence vectors using the above function for training and validation set - this will convert the full sentence 
#into a vector 

xtrain_IE_glove = [sent2vec(x, embeddings_index_glove) for x in X_train_IE_clean['posts']]
xvalid_IE_glove = [sent2vec(x, embeddings_index_glove) for x in X_validate_IE_clean['posts']]
 
xtrain_FT_glove = [sent2vec(x, embeddings_index_glove) for x in X_train_FT_clean['posts']]
xvalid_FT_glove = [sent2vec(x, embeddings_index_glove) for x in X_validate_FT_clean['posts']]

xtrain_IE_glove = np.array(xtrain_IE_glove)
xvalid_IE_glove = np.array(xvalid_IE_glove)

xtrain_FT_glove = np.array(xtrain_FT_glove)
xvalid_FT_glove = np.array(xvalid_FT_glove)

 
# Generate Word vectors of test data is_E
xtest_IE_glove = [sent2vec(x, embeddings_index_glove) for x in X_test_IE_clean['posts']]
xtest_IE_glove = np.array(xtest_IE_glove)

# Generate Word vectors of test data is_T
xtest_FT_glove = [sent2vec(x, embeddings_index_glove) for x in X_test_FT_clean['posts']]
xtest_FT_glove = np.array(xtest_FT_glove)

#print("xtest_IE_glove.shape = ", xtest_IE_glove.shape)
#xtrain_FT_glove[0]